In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

25/03/02 05:34:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
hdfs_path = '/olist_ka_data/'

In [3]:
olist_customers_df = spark.read.csv("/olist_ka_data/olist_customers_dataset.csv", header=True, inferSchema=True)
olist_geolocation_df = spark.read.csv("/olist_ka_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)
olist_order_items_df = spark.read.csv("/olist_ka_data/olist_order_items_dataset.csv", header=True, inferSchema=True)
olist_order_payments_df = spark.read.csv("/olist_ka_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
olist_order_reviews_df = spark.read.csv("/olist_ka_data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
olist_orders_df = spark.read.csv("/olist_ka_data/olist_orders_dataset.csv", header=True, inferSchema=True)
olist_products_df = spark.read.csv("/olist_ka_data/olist_products_dataset.csv", header=True, inferSchema=True)
olist_sellers_df = spark.read.csv("/olist_ka_data/olist_sellers_dataset.csv", header=True, inferSchema=True)
product_category_name_translation_df = spark.read.csv("/olist_ka_data/product_category_name_translation.csv", header=True, inferSchema=True)


In [4]:
from pyspark.sql.functions import *

In [5]:
def missing_values(df,df_name):
    print(f'Missing Values in {df_name} :')
    df.select([count(when(col(c).isNull(),1)).alias(c) for c in df.columns]).show()

In [6]:
missing_values(olist_customers_df,'customer Table')

Missing Values in customer Table :


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [7]:
missing_values(olist_orders_df,'Orders Table')

Missing Values in Orders Table :


+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [8]:
missing_values(olist_order_payments_df,'payments')

Missing Values in payments :
+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



In [ ]:
numerical vs continous

# Handle Missing Values

1. Drop missing Values
2. Fill missing Values ( for numerical value ) 
3. Impute Missing Values ( mean, median , knn) (for continous data)

In [9]:
olist_orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [10]:
olist_orders_df_cleaned = olist_orders_df.fillna({'order_delivered_customer_date':'9999-12-31'})

In [11]:
olist_orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

No, OLTP doesn't accept and errors out -

In [12]:
olist_order_payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [ ]:
from pyspark.sql import functions as F

payment_df_with_null = olist_order_payments_df.withColumn(
    'payment_value', 
    F.when(F.col('payment_value') != 99.33, F.col('payment_value'))
     .otherwise(F.lit(None))
)


In [17]:
payment_df_with_null =olist_order_payments_df.withColumn('payment_value', \
                                                         when(col('payment_value') !=99.33,\
                                                                  col('payment_value')).otherwise(lit(None)))

In [18]:
payment_df_with_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         null|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [20]:
from pyspark.ml.feature import Imputer

In [24]:
imputer  = Imputer(inputCols=['payment_value'],outputCols=['payment_value_imputed']).setStrategy('median')

In [25]:
payment_df_imputed = imputer.fit(payment_df_with_null).transform(payment_df_with_null)

In [26]:
payment_df_imputed.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         null|                100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               